# Complete BYOK Example App

> A full example FastHTML application demonstrating the BYOK system

In [1]:
#| default_exp examples.complete_app

## Setup and Imports

In [2]:
import os
from datetime import timedelta
from fasthtml.common import *
from fasthtml.jupyter import *

# BYOK Core imports
from cjm_fasthtml_byok.core.storage import BYOKManager
from cjm_fasthtml_byok.core.types import BYOKConfig, StorageBackend
from cjm_fasthtml_byok.middleware.beforeware import create_byok_beforeware, require_api_key
from cjm_fasthtml_byok.utils.helpers import get_key_summary, import_from_env, format_provider_name

# BYOK Component imports
from cjm_fasthtml_byok.components.forms import (
    KeyInputForm, 
    MultiProviderKeyForm,
    KeyManagementCard,
    KeyManagerDashboard,
    InlineKeyInput
)
from cjm_fasthtml_byok.components.alerts import (
    Alert,
    SecurityAlert,
    KeyStatusNotification,
    ValidationMessage,
    ToastContainer
)

# UI library imports
from cjm_fasthtml_daisyui.core.testing import create_theme_selector
from cjm_fasthtml_daisyui.core.resources import get_daisyui_headers
from cjm_fasthtml_daisyui.components.navigation.navbar import navbar, navbar_start, navbar_center, navbar_end
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_colors
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import gap, flex_display, flex_direction
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w, max_w, container
from cjm_fasthtml_tailwind.core.base import combine_classes

# OPTIONAL: Define custom provider configurations
# This is now optional - any provider name will work even without configuration
PROVIDER_CONFIG = {
    'main_llm': {
        'name': 'Main LLM Service',
        'placeholder': 'Enter your LLM API key',
        'key_prefix': 'llm-',
        'docs_url': 'https://docs.example.com/llm'
    },
    'embedding_service': {
        'name': 'Embedding Service',
        'placeholder': 'Your embedding API key',
        'key_prefix': 'emb-',
        'docs_url': 'https://docs.example.com/embeddings'
    },
    'vector_database': {
        'name': 'Vector Database',
        'placeholder': 'Database API key',
        'docs_url': 'https://docs.example.com/vectordb'
    }
    # Other providers will use auto-generated defaults
}

## Create the FastHTML Application

In [3]:
# Configuration
SECRET_KEY = os.environ.get("SECRET_KEY", "dev-secret-key-change-in-production")
DATABASE_URL = "sqlite:///byok_demo.db"  # Use SQLite file for demo

# Configure BYOK
byok_config = BYOKConfig(
    storage_backend=StorageBackend.HYBRID,  # Use both session and database
    default_ttl=timedelta(hours=24),
    auto_cleanup=True,
    require_https=False  # Disabled for demo
)

# Initialize BYOK manager with SQLAlchemy
byok = BYOKManager(
    secret_key=SECRET_KEY,
    db_url=DATABASE_URL,  # Pass database URL instead of db object
    config=byok_config
)

# Setup BYOK beforeware
def get_user_id(req):
    """Get user ID from session (mock for demo)"""
    # Access session from req.session, not req.scope['session']
    if hasattr(req, 'session'):
        return req.session.get("user_id", "demo-user")
    return "demo-user"

# Create the beforeware handler
byok_beforeware = create_byok_beforeware(byok)

# Create the FastHTML app with session support and beforeware
from starlette.middleware.sessions import SessionMiddleware

app, rt = fast_app(
    title="BYOK Demo",
    pico=False,
    hdrs=get_daisyui_headers(include_themes=True),
    secret_key=SECRET_KEY,
    sess_cls=SessionMiddleware,  # Use proper SessionMiddleware
    before=byok_beforeware,  # Add the beforeware here
    live=False
)

## Define Layout Components

In [4]:
def AppNavbar(req):
    """Create the application navbar"""
    from cjm_fasthtml_daisyui.components.actions.button import btn_styles
    from cjm_fasthtml_tailwind.utilities.typography import font_size
    
    return Div(
        Div(
            A("🔑 BYOK Demo", href="/", cls=combine_classes(btn, btn_styles.ghost, font_size.xl)),
            cls=str(navbar_start)
        ),
        Div(
            A("Dashboard", href="/", cls=combine_classes(btn, btn_styles.ghost)),
            A("Add Key", href="/add", cls=combine_classes(btn, btn_styles.ghost)),
            A("Settings", href="/settings", cls=combine_classes(btn, btn_styles.ghost)),
            create_theme_selector(),
            cls=str(navbar_end)
        ),
        cls=str(navbar)
    )

def PageLayout(req, title: str, *content, full_width=False):
    """Standard page layout
    
    Args:
        req: Request object
        title: Page title
        *content: Page content
        full_width: If True, don't wrap content in a container (for components with their own container)
    """
    if full_width:
        # Don't add container wrapper for components that manage their own layout
        page_content = Div(*content)
    else:
        # Standard container wrapper
        page_content = Div(
            *content,
            cls=combine_classes(
                container,
                m.x.auto,
                p(4),
                max_w.screen_xl
            )
        )
    
    return Titled(
        title,
        AppNavbar(req),
        page_content,
        ToastContainer(position="top", align="end")
    )

## Route Handlers

In [5]:
@rt("/")
def index(req, sess):
    """Main dashboard page"""
    # Set user ID in session
    sess["user_id"] = "demo-user"
    
    # Get stored keys summary
    # Mix of configured and unconfigured providers
    providers = ["main_llm", "embedding_service", "vector_database", 
                 "search_api", "analytics_tool", "monitoring_service"]
    user_id = get_user_id(req)
    
    # Check for any notifications
    notifications = []
    if msg := req.query_params.get("msg"):
        msg_type = req.query_params.get("type", "info")
        notifications.append(Alert(msg, kind=msg_type, dismissible=True))
    
    return PageLayout(
        req,
        Div(*notifications) if notifications else None,
        # KeyManagerDashboard has its own container, so don't double-wrap
        KeyManagerDashboard(
            req,
            providers=providers,
            byok_manager=byok,
            user_id=user_id,
            base_url="/api/keys",
            provider_config=PROVIDER_CONFIG  # Pass the optional config
        ),
        full_width=True  # Use full width since KeyManagerDashboard manages its own layout
    )

In [6]:
@rt("/add")
def add_key_page(req, sess):
    """Page to add a new API key"""
    from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight
    
    # Mix of configured and unconfigured providers
    providers = ["main_llm", "embedding_service", "vector_database", 
                 "search_api", "analytics_tool", "monitoring_service"]
    
    return PageLayout(
        req,
        "Add API Key",
        Div(
            H2("Add New API Key", cls=combine_classes(font_size._2xl, font_weight.bold, m.b(6))),
            Div(
                MultiProviderKeyForm(
                    providers=providers,
                    action="/api/keys/add",
                    provider_config=PROVIDER_CONFIG  # Pass the optional config
                ),
                cls=combine_classes(max_w.md, m.x.auto)
            )
        )
    )

In [7]:
@rt("/settings")
def settings_page(req, sess):
    """Settings page"""
    from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight, text_color, list_style, list_position
    
    user_id = get_user_id(req)
    summary = get_key_summary(byok, req, user_id, provider_config=PROVIDER_CONFIG)
    
    return PageLayout(
        req,
        "Settings",
        H2("Settings", cls=combine_classes(font_size._2xl, font_weight.bold, m.b(6))),
        
        # Security status
        Div(
            H3("Security Status", cls=combine_classes(font_size.xl, font_weight.semibold, m.b(4))),
            SecurityAlert(
                "HTTPS is not enabled. API keys may be transmitted insecurely.",
                severity="medium" if req.url.scheme == "http" else "low"
            ) if req.url.scheme == "http" else Alert(
                "Connection is secure (HTTPS)",
                kind="success"
            ),
            cls=str(m.b(8))
        ),
        
        # Key summary
        Div(
            H3("Stored Keys Summary", cls=combine_classes(font_size.xl, font_weight.semibold, m.b(4))),
            P(f"Total keys stored: {summary['count']}", cls=str(m.b(2))),
            Ul(
                *[
                    Li(
                        f"{key['display_name']}: {key['masked_key']} - {key['created']}"
                    )
                    for key in summary['keys']
                ],
                cls=combine_classes(list_style.disc, list_position.inside)
            ) if summary['keys'] else P("No keys stored", cls=str(text_color.gray(500))),
            cls=str(m.b(8))
        ),
        
        # Actions
        Div(
            H3("Actions", cls=combine_classes(font_size.xl, font_weight.semibold, m.b(4))),
            Form(
                Button(
                    "Clear All Keys",
                    type="submit",
                    cls=combine_classes(btn, btn_colors.error),
                    onclick="return confirm('Are you sure you want to delete all keys?');"
                ),
                method="post",
                action="/api/keys/clear-all"
            )
        )
    )

## API Endpoints

In [8]:
@rt("/api/keys/add", methods=["POST"])
def add_key(req, sess, provider: str, api_key: str):
    """Add a new API key"""
    user_id = get_user_id(req)
    
    try:
        # Store the key
        byok.set_key(
            req,
            provider=provider,
            api_key=api_key,
            user_id=user_id,
            ttl=timedelta(days=30)
        )
        
        return RedirectResponse(
            url="/?msg=API+key+added+successfully&type=success",
            status_code=303
        )
    except Exception as e:
        return RedirectResponse(
            url=f"/?msg=Failed+to+add+key:+{str(e)}&type=error",
            status_code=303
        )

In [9]:
@rt("/api/keys/{provider}", methods=["POST"])
def update_key(req, sess, provider: str, api_key: str):
    """Update an API key for a specific provider"""
    user_id = get_user_id(req)
    
    try:
        byok.set_key(
            req,
            provider=provider,
            api_key=api_key,
            user_id=user_id,
            ttl=timedelta(days=30)
        )
        
        # Get display name for the provider
        display_name = format_provider_name(provider, PROVIDER_CONFIG)
        
        return RedirectResponse(
            url=f"/?msg={display_name}+key+updated&type=success",
            status_code=303
        )
    except Exception as e:
        return RedirectResponse(
            url=f"/?msg=Failed+to+update+key:+{str(e)}&type=error",
            status_code=303
        )

In [10]:
@rt("/api/keys/{provider}/delete", methods=["POST"])
def delete_key(req, sess, provider: str):
    """Delete an API key"""
    user_id = get_user_id(req)
    
    try:
        byok.delete_key(req, provider, user_id)
        
        # Get display name for the provider
        display_name = format_provider_name(provider, PROVIDER_CONFIG)
        
        return RedirectResponse(
            url=f"/?msg={display_name}+key+deleted&type=info",
            status_code=303
        )
    except Exception as e:
        return RedirectResponse(
            url=f"/?msg=Failed+to+delete+key:+{str(e)}&type=error",
            status_code=303
        )

In [11]:
@rt("/api/keys/clear-all", methods=["POST"])
def clear_all_keys(req, sess):
    """Clear all stored API keys"""
    user_id = get_user_id(req)
    
    try:
        byok.clear_keys(req, user_id)
        
        return RedirectResponse(
            url="/?msg=All+keys+cleared&type=info",
            status_code=303
        )
    except Exception as e:
        return RedirectResponse(
            url=f"/?msg=Failed+to+clear+keys:+{str(e)}&type=error",
            status_code=303
        )

## Protected Routes Example

In [12]:
@rt("/demo/main_llm")
@require_api_key("main_llm", user_id_func=get_user_id)
def main_llm_demo(req, sess):
    """Demo route that requires Main LLM API key"""
    from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight
    
    user_id = get_user_id(req)
    provider = "main_llm"
    # Get the API key using the global byok manager
    api_key = byok.get_key(req, provider, user_id)
    
    # Get display name from config
    display_name = format_provider_name(provider, PROVIDER_CONFIG)
    
    return PageLayout(
        req,
        f"{display_name} Demo",
        H2(f"{display_name} Integration", cls=combine_classes(font_size._2xl, font_weight.bold, m.b(4))),
        Alert(
            f"Successfully accessed with API key: {api_key[:10]}...",
            kind="success"
        ),
        P(f"This page requires a {display_name} API key to access.", cls=str(m.t(4))),
        P("In a real application, you would use this key to make API calls to your LLM service.", cls=str(m.t(2)))
    )

@rt("/demo/analytics")
def analytics_demo(req, sess):
    """Demo page that works with any provider (no config needed)"""
    from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight
    
    user_id = get_user_id(req)
    provider = "analytics_tool"  # This provider has no config - uses defaults
    
    # Check if key exists
    has_key = byok.has_key(req, provider, user_id)
    
    if not has_key:
        return PageLayout(
            req,
            "Analytics Demo",
            H2("Analytics Tool Demo", cls=combine_classes(font_size._2xl, font_weight.bold, m.b(4))),
            Alert(
                "No Analytics Tool key configured. This provider works without predefined configuration!",
                kind="warning"
            ),
            Div(
                KeyInputForm(
                    provider=provider,
                    action=f"/api/keys/{provider}",
                    provider_config=PROVIDER_CONFIG  # Will use auto-generated defaults
                ),
                cls=str(max_w.md)
            )
        )
    
    api_key = byok.get_key(req, provider, user_id)
    return PageLayout(
        req,
        "Analytics Demo",
        H2("Analytics Tool Integration", cls=combine_classes(font_size._2xl, font_weight.bold, m.b(4))),
        Alert(
            f"Successfully configured! Key: {api_key[:10] if api_key else 'N/A'}...",
            kind="success"
        ),
        P("This demonstrates that any provider name works, even without configuration.", cls=str(m.t(4)))
    )

## Run the Application

In [13]:
#| eval: false
# For Jupyter notebook display
from fasthtml.jupyter import *
from cjm_fasthtml_daisyui.core.testing import start_test_server

# Start the server
server = start_test_server(app, port=5001)

# Display in Jupyter
display(HTMX(port=server.port))

print("🚀 BYOK Demo App is running!")
print(f"📍 Visit http://localhost:{server.port} to see the app")
print("\nTest flow:")
print("1. Visit the dashboard - shows mix of configured and unconfigured providers")
print("2. Click 'Add Key' to add a new API key")
print("3. Notice how configured providers (main_llm, embedding_service, vector_database)")
print("   show custom names and placeholders")
print("4. Unconfigured providers (search_api, analytics_tool, monitoring_service)")
print("   work perfectly with auto-generated defaults")
print("5. Try /demo/main_llm (requires configured provider key)")
print("6. Try /demo/analytics (works with unconfigured provider)")
print("7. Check Settings page for summary with proper display names")
print("\nPress Stop button in Jupyter to stop the server")

🚀 BYOK Demo App is running!
📍 Visit http://localhost:5001 to see the app

Test flow:
1. Visit the dashboard - shows mix of configured and unconfigured providers
2. Click 'Add Key' to add a new API key
3. Notice how configured providers (main_llm, embedding_service, vector_database)
   show custom names and placeholders
4. Unconfigured providers (search_api, analytics_tool, monitoring_service)
   work perfectly with auto-generated defaults
5. Try /demo/main_llm (requires configured provider key)
6. Try /demo/analytics (works with unconfigured provider)
7. Check Settings page for summary with proper display names

Press Stop button in Jupyter to stop the server


In [14]:
#| eval: false
# Stop the server when done
server.stop()

## Alternative: Run as Standalone Script

In [15]:
#| eval: false
# To run as a standalone script, uncomment the following:
# if __name__ == "__main__":
#     import uvicorn
#     uvicorn.run(app, host="0.0.0.0", port=5001)

## Testing Different Scenarios

### Test Scenarios to Try:

1. **Mixed Providers**: Dashboard shows both configured providers (with custom names) and unconfigured ones (with auto-generated names)
2. **Add Keys**: Add keys for any provider - custom or standard
3. **Update Keys**: Change existing keys for any provider
4. **Delete Keys**: Remove individual keys
5. **Protected Routes**: Try accessing `/demo/main_llm` with and without a key
6. **Unconfigured Provider**: Try `/demo/analytics` to see how unconfigured providers work
7. **Security Alerts**: Check the security warnings on the Settings page
8. **Clear All**: Use the "Clear All Keys" button in Settings
9. **Session Persistence**: Keys persist during the session
10. **Database Persistence**: Keys persist across sessions (using SQLite)

### Key Features Demonstrated:

- ✅ **Provider-agnostic design** - works with ANY provider name
- ✅ **Optional configuration** - customize providers when needed
- ✅ **Auto-generated defaults** - sensible defaults for unknown providers
- ✅ Secure key storage with encryption
- ✅ Session and database hybrid storage
- ✅ Protected routes with `@require_api_key` decorator
- ✅ Visual feedback with alerts and notifications
- ✅ Responsive dashboard layout
- ✅ Security warnings and status
- ✅ Key management (CRUD operations)
- ✅ Clean UI with daisyUI and Tailwind CSS

### Configuration Example:

```python
# Optional provider configuration
PROVIDER_CONFIG = {
    'my_service': {
        'name': 'My Custom Service',
        'placeholder': 'Enter your service key',
        'key_prefix': 'srv-',
        'docs_url': 'https://docs.example.com'
    }
}

# Pass to components
form = KeyInputForm(
    provider="my_service",
    provider_config=PROVIDER_CONFIG
)
```

### Without Configuration:

```python
# Works automatically without any config!
form = KeyInputForm(provider="any_provider_name")
# Generates: name="Any Provider Name"
#           placeholder="Enter your Any Provider Name API key"
```